# G1

## Dependências

In [1]:
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 30.8 MB/s eta 0:00:00


## Coleta de Dados

In [2]:
# ----------------------------------------------------------------------
# PASSO 1: INSTALAÇÃO DE DEPENDÊNCIAS DO SCRAPING
# ----------------------------------------------------------------------
print("--- [SETUP] Instalando dependências do sistema e do Python ---")
!apt-get update -qq > /dev/null
!apt-get install -y libvulkan1 libglib2.0-0 libnss3 libgconf-2-4 libfontconfig1 -qq > /dev/null
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb > /dev/null
!pip install pandas beautifulsoup4 selenium webdriver-manager pytz -q
print("--- [SETUP] Instalação concluída ---")


# ----------------------------------------------------------------------
# PASSO 2: IMPORTAÇÕES E CONFIGURAÇÃO
# ----------------------------------------------------------------------
import pandas as pd
from datetime import datetime, timedelta
import time
from urllib.parse import urljoin, quote
from bs4 import BeautifulSoup
import re
import pytz

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

print("\n--- [INIT] Fase 1: Bibliotecas importadas ---")

BR_TIMEZONE = pytz.timezone('America/Sao_Paulo')
query_busca = "defesa civil campinas piracicaba"
URL_G1_BASE = f"https://g1.globo.com/busca/?q={quote(query_busca)}&order=recent&period=7d"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
dados_coletados = []


# ----------------------------------------------------------------------
# PASSO 3: FUNÇÕES DE SCRAPING (APRIMORADAS)
# ----------------------------------------------------------------------

def setup_driver():
    """Configura o WebDriver com opções anti-detecção."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent={HEADERS["User-Agent"]}')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def save_diagnostics(driver, filename_prefix):
    """Salva screenshot e HTML para depuração."""
    driver.save_screenshot(f"{filename_prefix}_captura.png")
    with open(f"{filename_prefix}_pagina.html", 'w', encoding='utf-8') as f:
        f.write(driver.page_source)
    print(f"--- [DIAGNÓSTICO] Arquivos de erro salvos como '{filename_prefix}_...'")

def parse_relative_date(date_str, timezone):
    """
    Converte strings como 'há 3 horas' ou '19/10/2025' em um datetime.
    VERSÃO APRIMORADA com mais formatos e debug.
    """
    if not date_str:
        print(f"[PARSE_DATE] Data vazia recebida")
        return None

    now = datetime.now(timezone)
    date_str_lower = date_str.lower().strip()

    # Debug: mostra o que está sendo parseado
    print(f"[PARSE_DATE] Tentando parsear: '{date_str_lower}'")

    try:
        # Formato: "agora mesmo", "agora"
        if 'agora' in date_str_lower:
            return now

        # Formato: "há X minuto(s)"
        if 'minuto' in date_str_lower:
            match = re.search(r'(\d+)', date_str)
            if match:
                minutes = int(match.group(1))
                return now - timedelta(minutes=minutes)

        # Formato: "há X hora(s)"
        if 'hora' in date_str_lower:
            match = re.search(r'(\d+)', date_str)
            if match:
                hours = int(match.group(1))
                return now - timedelta(hours=hours)

        # Formato: "ontem"
        if 'ontem' in date_str_lower:
            return now - timedelta(days=1)

        # Formato: "há X dia(s)"
        if 'dia' in date_str_lower and 'há' in date_str_lower:
            match = re.search(r'(\d+)', date_str)
            if match:
                days = int(match.group(1))
                return now - timedelta(days=days)

        # Formato: "DD/MM/YYYY"
        if '/' in date_str:
            parsed_date = datetime.strptime(date_str_lower, '%d/%m/%Y')
            return timezone.localize(parsed_date)

        # Formato: "DD/MM/YYYY HH:MM" ou "DD/MM/YY"
        date_patterns = [
            r'(\d{2})/(\d{2})/(\d{4})',  # DD/MM/YYYY
            r'(\d{2})/(\d{2})/(\d{2})',   # DD/MM/YY
        ]
        for pattern in date_patterns:
            match = re.search(pattern, date_str)
            if match:
                try:
                    if len(match.group(3)) == 2:  # Ano com 2 dígitos
                        parsed_date = datetime.strptime(f"{match.group(1)}/{match.group(2)}/20{match.group(3)}", '%d/%m/%Y')
                    else:
                        parsed_date = datetime.strptime(f"{match.group(1)}/{match.group(2)}/{match.group(3)}", '%d/%m/%Y')
                    return timezone.localize(parsed_date)
                except ValueError:
                    continue

        print(f"[PARSE_DATE] Formato não reconhecido: '{date_str_lower}'")
        return None

    except Exception as e:
        print(f"[PARSE_DATE] Erro ao parsear data '{date_str}': {e}")
        return None

def extract_date_from_element(noticia, timezone):
    """
    Tenta múltiplas estratégias para extrair a data de publicação.
    NOVA FUNÇÃO para ser mais agressiva na busca.
    """
    date_selectors = [
        ('div', 'widget--info__meta'),
        ('span', 'widget--info__meta'),
        ('time', None),
        ('div', 'feed-post-datetime'),
        ('span', 'feed-post-datetime'),
    ]

    for tag, class_name in date_selectors:
        try:
            if class_name:
                date_element = noticia.find(tag, class_=class_name)
            else:
                date_element = noticia.find(tag)

            if date_element:
                # Tenta pegar o atributo datetime primeiro (mais confiável)
                if date_element.has_attr('datetime'):
                    date_str = date_element['datetime']
                    print(f"[EXTRACT_DATE] Encontrado datetime attribute: '{date_str}'")
                    try:
                        return datetime.fromisoformat(date_str.replace('Z', '+00:00')).astimezone(timezone)
                    except:
                        pass

                # Se não tiver datetime, pega o texto
                date_str = date_element.text.strip()
                if date_str:
                    print(f"[EXTRACT_DATE] Encontrado texto de data: '{date_str}'")
                    parsed_date = parse_relative_date(date_str, timezone)
                    if parsed_date:
                        return parsed_date
        except Exception as e:
            print(f"[EXTRACT_DATE] Erro ao extrair de {tag}.{class_name}: {e}")
            continue

    # Se nenhum seletor funcionou, tenta buscar qualquer texto que pareça uma data
    text_content = noticia.get_text()
    date_patterns = [
        r'há\s+\d+\s+(minuto|hora|dia)s?',
        r'\d{2}/\d{2}/\d{4}',
        r'ontem',
        r'agora'
    ]

    for pattern in date_patterns:
        match = re.search(pattern, text_content, re.IGNORECASE)
        if match:
            date_str = match.group(0)
            print(f"[EXTRACT_DATE] Encontrado via regex: '{date_str}'")
            parsed_date = parse_relative_date(date_str, timezone)
            if parsed_date:
                return parsed_date

    print(f"[EXTRACT_DATE] Nenhuma data encontrada para esta notícia")
    return None

def scrape_g1(base_url):
    """
    Versão aprimorada com melhor extração de datas.
    """
    print(f"\n[G1] Coletando da URL base: {base_url}")
    driver = setup_driver()
    wait = WebDriverWait(driver, 15)

    NUMERO_DE_PAGINAS_TOTAIS = 6
    noticias_coletadas_total = 0

    try:
        for page_num in range(1, NUMERO_DE_PAGINAS_TOTAIS + 1):
            page_url = f"{base_url}&page={page_num}"
            print(f"\n[G1] === Coletando dados da Página {page_num} ===")

            if page_num > 1:
                old_page_html = driver.find_element(By.TAG_NAME, "html")

            driver.get(page_url)

            if page_num == 1:
                try:
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "cookie-banner-lgpd-accept"))).click()
                    print("[G1] Banner de cookies aceito.")
                except TimeoutException:
                    print("[G1] Banner de cookies não encontrado ou já aceito.")

            if page_num > 1:
                print(f"[G1] Esperando a página {page_num-1} desaparecer...")
                wait.until(EC.staleness_of(old_page_html))

            content_selector_css = "li.widget--info"
            print(f"[G1] Esperando conteúdo da página {page_num}...")
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, content_selector_css)))

            html_content = driver.page_source
            soup = BeautifulSoup(html_content, 'html.parser')
            noticias_na_pagina = soup.select(content_selector_css)

            if not noticias_na_pagina:
                print("[G1] Página sem notícias (ou seletor mudou). Interrompendo paginação.")
                break

            print(f"[G1] Encontradas {len(noticias_na_pagina)} notícias nesta página")

            for idx, noticia in enumerate(noticias_na_pagina, 1):
                print(f"\n[G1] --- Processando notícia {idx}/{len(noticias_na_pagina)} ---")

                text_container = noticia.find('div', class_='widget--info__text-container')
                if not text_container:
                    print("[G1] Container de texto não encontrado")
                    continue

                link_tag = text_container.find('a')
                if not link_tag:
                    print("[G1] Link não encontrado")
                    continue

                titulo_tag = link_tag.find('div', class_='widget--info__title')
                resumo_tag = link_tag.find('p', class_='widget--info__description')

                # MUDANÇA CRÍTICA: Usa a nova função de extração
                data_publicacao = extract_date_from_element(noticia, BR_TIMEZONE)

                item = {
                    'fonte': 'G1',
                    'titulo': titulo_tag.text.strip() if titulo_tag else 'N/A',
                    'resumo': resumo_tag.text.strip() if resumo_tag else 'N/A',
                    'link': link_tag['href'],
                    'horario_coleta': datetime.now(BR_TIMEZONE).isoformat(),
                    'data_publicacao': data_publicacao
                }

                print(f"[G1] ✓ Título: {item['titulo'][:50]}...")
                print(f"[G1] ✓ Data publicação: {data_publicacao}")

                dados_coletados.append(item)

            noticias_coletadas_pagina = len(noticias_na_pagina)
            noticias_coletadas_total += noticias_coletadas_pagina
            print(f"\n[G1] Página {page_num} concluída. {noticias_coletadas_pagina} notícias adicionadas.")
            time.sleep(1)

    except TimeoutException:
        print(f"[G1] Condição de 'sem dados': Página {page_num} não carregou ou estava vazia. Encerrando coleta.")
    except Exception as e:
        print(f"[G1] ERRO INESPERADO: {e}")
        save_diagnostics(driver, f"g1_erro_page_{page_num}")
    finally:
        driver.quit()
        print(f"\n[G1] Coleta finalizada. Total de {noticias_coletadas_total} notícias extraídas.")


def executar_coleta_automatizada():
    print("\n--- [RUN] Iniciando Coleta Automatizada ---")
    dados_coletados.clear()
    scrape_g1(URL_G1_BASE)
    print("\n--- [RUN] Coleta finalizada ---")

    if dados_coletados:
        df = pd.DataFrame(dados_coletados)
        df = df.drop_duplicates(subset=['link'], keep='first')

        # Estatísticas de captura de datas
        total = len(df)
        com_data = df['data_publicacao'].notna().sum()
        sem_data = total - com_data

        print(f"\n--- [ESTATÍSTICAS] ---")
        print(f"Total de notícias: {total}")
        print(f"Com data de publicação: {com_data} ({com_data/total*100:.1f}%)")
        print(f"Sem data de publicação: {sem_data} ({sem_data/total*100:.1f}%)")

        return df
    else:
        return pd.DataFrame()

# ----------------------------------------------------------------------
# PASSO 4: EXECUÇÃO DA COLETA
# ----------------------------------------------------------------------

df_output = executar_coleta_automatizada()

if not df_output.empty:
    print("\n--- [OUTPUT] Coleta concluída. Exibindo 5 primeiros resultados: ---")
    try:
        from IPython.display import display
        display(df_output.head())
    except (ImportError, NameError):
        print(df_output.head())
else:
    print("\n--- [OUTPUT] Nenhuma notícia nova foi coletada. ---")

--- [SETUP] Instalando dependências do sistema e do Python ---
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libatk-bridge2.0-0 (>= 2.5.3); however:
  Package libatk-bridge2.0-0 is not installed.
 google-chrome-stable depends on libatk1.0-0 (>= 2.11.90); however:
  Package libatk1.0-0 is not installed.
 google-chrome-stable depends on libatspi2.0-0 (>= 2.9.90); however:
  Package libatspi2.0-0 is not installed.
 google-chrome-stable depends on libxcomposite1 (>= 1:0.4.4-1); however:
  Package libxcomposite1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Errors were encountered while processing:
 google-chrome-stable
--- [SETUP] Instalação concluída ---

--- [INI

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5be0853074e3 <unknown>
#1 0x5be085036c76 <unknown>
#2 0x5be08505fd78 <unknown>
#3 0x5be08505c029 <unknown>
#4 0x5be08509accc <unknown>
#5 0x5be08509a47f <unknown>
#6 0x5be085091de3 <unknown>
#7 0x5be0850672dd <unknown>
#8 0x5be08506834e <unknown>
#9 0x5be0852c73e4 <unknown>
#10 0x5be0852cb3d7 <unknown>
#11 0x5be0852d5b20 <unknown>
#12 0x5be0852cc023 <unknown>
#13 0x5be08529a1aa <unknown>
#14 0x5be0852f06b8 <unknown>
#15 0x5be0852f0847 <unknown>
#16 0x5be085300243 <unknown>
#17 0x79eb98837ac3 <unknown>


## Limpeza e Estruturação

In [ ]:
import pandas as pd
import re
from datetime import datetime

# Verifique se o DataFrame 'df_output' não está vazio antes de processar
if 'df_output' in locals() and not df_output.empty:
    def classificar_risco(row):
        """
        Analisa o título e o resumo de uma notícia para atribuir um nível de risco.
        """
        # Consolida o texto em um único campo e converte para minúsculas
        texto_completo = f"{row['titulo']} {row['resumo']}".lower()

        # Nível 5 (Crítico): Risco iminente e de grande escala
        if re.search(r'estiagem|enchente|cheia', texto_completo):
            return 5

        # Nível 4 (Alto): Impacto direto em sistemas críticos
        if re.search(r'cantareira|qualidade da [aá]gua', texto_completo):
            return 4

        # Nível 3 (Médio): Eventos meteorológicos severos
        if re.search(r'tempestade|chuva forte', texto_completo):
            return 3

        # Nível 2 (Baixo): Eventos meteorológicos comuns
        if re.search(r'chuva', texto_completo):
            return 2

        # Nível 1 (Atenção): Eventos de menor impacto potencial
        if re.search(r'vento|ventania', texto_completo):
            return 1

        # Nível 0 (Informativo): Nenhum risco operacional identificado
        return 0

    print("--- [ANÁLISE] Iniciando a classificação de risco das notícias coletadas ---")

    # Aplica a função de classificação para criar a nova coluna 'nivel_risco'
    df_output['nivel_risco'] = df_output.apply(classificar_risco, axis=1)

    # Ordena o DataFrame pelo nível de risco, do mais alto para o mais baixo
    df_priorizado = df_output.sort_values(by='nivel_risco', ascending=False)

    print("--- [ANÁLISE] Classificação concluída. Exibindo resultados priorizados: ---")

    # --- MUDANÇA: Adicionando 'data_publicacao' à exibição ---
    colunas_exibicao = ['nivel_risco', 'fonte', 'data_publicacao', 'titulo', 'resumo', 'link', 'horario_coleta']

    try:
        from IPython.display import display
        # Exibe o DataFrame com a nova coluna de data de publicação
        display(df_priorizado[colunas_exibicao])
    except (ImportError, NameError):
        print(df_priorizado[colunas_exibicao])
else:
    print("--- [ANÁLISE] 'df_output' está vazio. Pulando etapa de classificação. ---")

## Sync GCP BigQuery

In [ ]:
# Adicione esta linha no início do seu script, junto com os outros !pip
!pip install google-cloud-bigquery pandas-gbq -q

In [ ]:
# ----------------------------------------------------------------------
# PASSO 1: INSTALAR BIBLIOTECAS (se ainda não tiver feito)
# ----------------------------------------------------------------------
!pip install google-cloud-bigquery pandas-gbq -q

# ----------------------------------------------------------------------
# PASSO 2: AUTENTICAR E FAZER UPLOAD
# ----------------------------------------------------------------------
import pandas as pd
from google.colab import auth
import pandas_gbq
from google.cloud import bigquery # Precisamos do cliente completo do BigQuery para o MERGE

# Autentica o usuário para permitir o acesso ao Google Cloud
print("--- [AUTH] Autenticando para acesso ao BigQuery ---")
auth.authenticate_user()
print("--- [AUTH] Autenticação concluída com sucesso ---")


# Verifique se o DataFrame df_priorizado existe e não está vazio
if 'df_priorizado' in locals() and not df_priorizado.empty:
    print("\n--- [BIGQUERY] Iniciando processo de upload ---")

    # 1. Defina os nomes do projeto e das tabelas
    project_id = "bacias-pcj"
    main_table_id = "scrapping.noticias"        # Tabela principal, de produção
    staging_table_id = "scrapping.noticias_staging" # Tabela de rascunho/temporária

    # 2. Crie uma cópia para evitar alterar o DataFrame original
    df_to_upload = df_priorizado.copy()

    # 3. Garanta que as colunas de data estão no formato correto
    df_to_upload['horario_coleta'] = pd.to_datetime(df_to_upload['horario_coleta'])
    df_to_upload['data_publicacao'] = pd.to_datetime(df_to_upload['data_publicacao'])

    # 4. Defina o schema (estrutura) da tabela
    table_schema = [
        {'name': 'nivel_risco', 'type': 'INTEGER'},
        {'name': 'fonte', 'type': 'STRING'},
        {'name': 'titulo', 'type': 'STRING'},
        {'name': 'resumo', 'type': 'STRING'},
        {'name': 'link', 'type': 'STRING'},
        {'name': 'horario_coleta', 'type': 'TIMESTAMP'},
        {'name': 'data_publicacao', 'type': 'TIMESTAMP'}
    ]

    try:
        # --- ETAPA A: UPLOAD PARA A TABELA DE STAGING (RASCUNHO) ---
        print(f"Enviando {len(df_to_upload)} registros para a tabela de staging: '{staging_table_id}'...")
        pandas_gbq.to_gbq(
            df_to_upload,
            destination_table=staging_table_id,
            project_id=project_id,
            if_exists='replace',
            table_schema=table_schema
        )
        print("Upload para staging concluído com sucesso.")

        # --- ETAPA B: EXECUTAR O MERGE NA TABELA PRINCIPAL ---
        print(f"Executando MERGE da staging para a tabela principal: '{main_table_id}'...")

        client = bigquery.Client(project=project_id)

        # --- MUDANÇA CRÍTICA: Adicionando a lógica WHEN MATCHED ---
        merge_sql = f"""
        MERGE `{project_id}.{main_table_id}` T
        USING `{project_id}.{staging_table_id}` S
        ON T.link = S.link

        -- Se o link já existe, MAS a data_publicacao está NULA, atualize-a.
        WHEN MATCHED AND T.data_publicacao IS NULL THEN
          UPDATE SET T.data_publicacao = S.data_publicacao

        -- Se o link é totalmente novo, insira a linha.
        WHEN NOT MATCHED BY TARGET THEN
          INSERT (nivel_risco, fonte, titulo, resumo, link, horario_coleta, data_publicacao)
          VALUES(S.nivel_risco, S.fonte, S.titulo, S.resumo, S.link, S.horario_coleta, S.data_publicacao)
        """
        # --- FIM DA MUDANÇA ---

        query_job = client.query(merge_sql)
        query_job.result()  # Espera a consulta terminar

        print(f"--- [BIGQUERY] MERGE concluído! {query_job.num_dml_affected_rows} linhas foram afetadas (inseridas ou atualizadas).")
        print(f"--- [BIGQUERY] Sucesso! Pipeline de upload finalizado.")

    except Exception as e:
        print(f"--- [BIGQUERY] ERRO: Falha no pipeline de upload. Detalhes: {e}")
else:
    print("\n--- [BIGQUERY] DataFrame 'df_priorizado' não encontrado ou está vazio. Nenhum dado para enviar. ---")

# Defesa Civil - Piracicaba

## Constantes


In [ ]:
BR_TIMEZONE = pytz.timezone('America/Sao_Paulo')
URL_BASE = "https://piracicaba.sp.gov.br/noticias/?competencia=defesa-civil"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
dados_coletados = []

## Scraping Functions

In [ ]:
def setup_driver():
    """Configura o WebDriver com opções anti-detecção."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent={HEADERS["User-Agent"]}')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def parse_date_piracicaba(date_str, timezone):
    """
    Converte data do formato do site de Piracicaba (DD/MM/YYYY) em datetime.
    """
    if not date_str:
        print(f"[PARSE_DATE] Data vazia recebida")
        return None

    date_str_clean = date_str.strip()
    print(f"[PARSE_DATE] Tentando parsear: '{date_str_clean}'")

    try:
        # Formato esperado: DD/MM/YYYY
        if '/' in date_str_clean:
            # Remove qualquer texto extra e pega apenas a data
            match = re.search(r'(\d{2}/\d{2}/\d{4})', date_str_clean)
            if match:
                date_only = match.group(1)
                parsed_date = datetime.strptime(date_only, '%d/%m/%Y')
                return timezone.localize(parsed_date)

        print(f"[PARSE_DATE] Formato não reconhecido: '{date_str_clean}'")
        return None

    except Exception as e:
        print(f"[PARSE_DATE] Erro ao parsear data '{date_str_clean}': {e}")
        return None

def scrape_defesa_civil_piracicaba(base_url):
    """
    Scraping específico para o site da Defesa Civil de Piracicaba.
    Coleta múltiplas páginas de notícias.
    """
    print(f"\n[DC_PIRACICABA] Coletando da URL: {base_url}")
    driver = setup_driver()
    wait = WebDriverWait(driver, 15)

    noticias_coletadas_total = 0
    NUMERO_DE_PAGINAS = 5  # Há 5 páginas de notícias

    try:
        for page_num in range(1, NUMERO_DE_PAGINAS + 1):
            # Monta a URL da página
            if page_num == 1:
                page_url = base_url
            else:
                page_url = f"https://piracicaba.sp.gov.br/noticias/page/{page_num}/?competencia=defesa-civil"

            print(f"\n[DC_PIRACICABA] === Coletando PÁGINA {page_num}/{NUMERO_DE_PAGINAS} ===")
            print(f"[DC_PIRACICABA] URL: {page_url}")

            driver.get(page_url)
            print("[DC_PIRACICABA] Página carregada. Aguardando conteúdo...")

            # Aguarda o carregamento das notícias
            time.sleep(3)

            html_content = driver.page_source
            soup = BeautifulSoup(html_content, 'html.parser')

            # Encontrar todas as notícias
            # Estrutura: div.card dentro de div.col-12
            noticias_na_pagina = soup.select('div.col-12 > div.card')

            if not noticias_na_pagina:
                print(f"[DC_PIRACICABA] Nenhuma notícia encontrada na página {page_num}. Encerrando coleta.")
                break

            print(f"[DC_PIRACICABA] Encontradas {len(noticias_na_pagina)} notícias na página {page_num}")

            for idx, card in enumerate(noticias_na_pagina, 1):
                print(f"[DC_PIRACICABA] --- Processando notícia {idx}/{len(noticias_na_pagina)} (Página {page_num}) ---")

            try:
                # Link: dentro de a.stretched-link
                link_tag = card.select_one('a.stretched-link')
                if not link_tag or not link_tag.get('href'):
                    print("[DC_PIRACICABA] Link não encontrado")
                    continue

                link = link_tag['href']
                # Garantir URL absoluta
                if not link.startswith('http'):
                    link = f"https://piracicaba.sp.gov.br{link}"

                # Título: dentro de div.card-title h4
                titulo_tag = card.select_one('div.card-title')
                titulo = titulo_tag.text.strip() if titulo_tag else 'N/A'

                # Resumo: dentro de div.card-text p
                resumo_tag = card.select_one('div.card-text p')
                resumo = resumo_tag.text.strip() if resumo_tag else 'N/A'

                # Data: no final, dentro de div.card-text.text-muted com ícone de calendário
                data_tag = card.select_one('div.card-text.text-muted.small')
                data_publicacao = None

                if data_tag:
                    # Extrai apenas o texto da data (remove o ícone)
                    date_text = data_tag.get_text(strip=True)
                    # Remove o texto do ícone
                    date_text = date_text.replace('Data da publicação', '').strip()
                    print(f"[DC_PIRACICABA] Data encontrada (texto): '{date_text}'")
                    data_publicacao = parse_date_piracicaba(date_text, BR_TIMEZONE)

                item = {
                    'fonte': 'DEFESA_CIVIL_PIRACICABA',
                    'titulo': titulo,
                    'resumo': resumo,
                    'link': link,
                    'horario_coleta': datetime.now(BR_TIMEZONE).isoformat(),
                    'data_publicacao': data_publicacao
                }

                print(f"[DC_PIRACICABA] ✓ Título: {titulo[:60]}...")
                print(f"[DC_PIRACICABA] ✓ Data: {data_publicacao}")
                print(f"[DC_PIRACICABA] ✓ Link: {link}")

                dados_coletados.append(item)
                noticias_coletadas_total += 1

            except Exception as e:
                print(f"[DC_PIRACICABA] Erro ao processar notícia {idx}: {e}")
                import traceback
                traceback.print_exc()
                continue

            print(f"\n[DC_PIRACICABA] Página {page_num} concluída. {len(noticias_na_pagina)} notícias processadas.")
            time.sleep(2)  # Pausa entre páginas

        print(f"\n[DC_PIRACICABA] Coleta concluída. Total: {noticias_coletadas_total} notícias extraídas.")

    except Exception as e:
        print(f"[DC_PIRACICABA] ERRO INESPERADO: {e}")
        import traceback
        traceback.print_exc()
    finally:
        driver.quit()

    return dados_coletados


def executar_coleta_dc_piracicaba():
    print("\n=== [RUN] INICIANDO COLETA DEFESA CIVIL PIRACICABA ===")
    dados_coletados.clear()
    scrape_defesa_civil_piracicaba(URL_BASE)
    print("\n=== [RUN] COLETA FINALIZADA ===")

    if dados_coletados:
        df = pd.DataFrame(dados_coletados)
        df = df.drop_duplicates(subset=['link'], keep='first')

        total = len(df)
        com_data = df['data_publicacao'].notna().sum()
        sem_data = total - com_data

        print(f"\n--- [ESTATÍSTICAS] ---")
        print(f"Total de notícias: {total}")
        print(f"Com data de publicação: {com_data} ({com_data/total*100:.1f}%)")
        print(f"Sem data de publicação: {sem_data} ({sem_data/total*100:.1f}%)")

        return df
    else:
        return pd.DataFrame()


## Collection

In [ ]:
df_output = executar_coleta_dc_piracicaba()

if not df_output.empty:
    print("\n--- [OUTPUT] Coleta concluída. Exibindo primeiros resultados: ---")
    try:
        from IPython.display import display
        display(df_output.head(10))
    except (ImportError, NameError):
        print(df_output.head(10))
else:
    print("\n--- [OUTPUT] Nenhuma notícia foi coletada. ---")


=== [RUN] INICIANDO COLETA DEFESA CIVIL PIRACICABA ===

[DC_PIRACICABA] Coletando da URL: https://piracicaba.sp.gov.br/noticias/?competencia=defesa-civil

[DC_PIRACICABA] === Coletando PÁGINA 1/5 ===
[DC_PIRACICABA] URL: https://piracicaba.sp.gov.br/noticias/?competencia=defesa-civil
[DC_PIRACICABA] Página carregada. Aguardando conteúdo...
[DC_PIRACICABA] Encontradas 10 notícias na página 1
[DC_PIRACICABA] --- Processando notícia 1/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 2/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 3/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 4/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 5/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 6/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 7/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 8/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 9/10 (Página 1) ---
[DC_PIRACICABA] --- Processando notícia 10/10 (Página 1) ---
[DC_

,fonte,titulo,resumo,link,horario_coleta,data_publicacao
0,DEFESA_CIVIL_PIRACICABA,Maio Amarelo: simulado alerta sobre gravidade ...,Ação aconteceu no Centro da cidade na manhã de...,https://piracicaba.sp.gov.br/noticias/maio-ama...,2025-10-28T21:01:24.439354-03:00,2025-05-22 00:00:00-03:00
1,DEFESA_CIVIL_PIRACICABA,Piracicaba cria Comitê Florestal para combate ...,Encontro reuniu representantes do poder Execut...,https://piracicaba.sp.gov.br/noticias/piracica...,2025-10-28T21:01:33.905474-03:00,2024-09-06 00:00:00-03:00
2,DEFESA_CIVIL_PIRACICABA,Prefeitura realiza limpeza de comportas e Cana...,Foram retirados aproximadamente dois caminhões...,https://piracicaba.sp.gov.br/noticias/prefeitu...,2025-10-28T21:01:40.798693-03:00,2024-01-15 00:00:00-03:00
3,DEFESA_CIVIL_PIRACICABA,Temporal causou queda de mais de 30 árvores; f...,"Em uma semana, ventos derrubaram mais de 160 á...",https://piracicaba.sp.gov.br/noticias/temporal...,2025-10-28T21:01:48.175198-03:00,2023-10-05 00:00:00-03:00
4,DEFESA_CIVIL_PIRACICABA,Prefeitura realiza força-tarefa para minimizar...,Equipes trabalham desde ontem; bairros mais at...,https://piracicaba.sp.gov.br/noticias/prefeitu...,2025-10-28T21:01:54.881009-03:00,2023-09-28 00:00:00-03:00


## Sync BQ

In [ ]:
if not df_output.empty:
    print("\n=== [BIGQUERY] INICIANDO SINCRONIZAÇÃO ===")

    from google.colab import auth
    import pandas_gbq
    from google.cloud import bigquery

    # Autentica
    print("--- [AUTH] Autenticando para acesso ao BigQuery ---")
    auth.authenticate_user()
    print("--- [AUTH] Autenticação concluída ---")

    # Configurações
    project_id = "bacias-pcj"
    main_table_id = "scrapping.dc_piracicaba"
    staging_table_id = "scrapping.dc_piracicaba_staging"

    # Prepara DataFrame
    df_to_upload = df_output.copy()
    df_to_upload['horario_coleta'] = pd.to_datetime(df_to_upload['horario_coleta'])
    df_to_upload['data_publicacao'] = pd.to_datetime(df_to_upload['data_publicacao'])

    # Schema da tabela
    table_schema = [
        {'name': 'fonte', 'type': 'STRING'},
        {'name': 'titulo', 'type': 'STRING'},
        {'name': 'resumo', 'type': 'STRING'},
        {'name': 'link', 'type': 'STRING'},
        {'name': 'horario_coleta', 'type': 'TIMESTAMP'},
        {'name': 'data_publicacao', 'type': 'TIMESTAMP'}
    ]

    try:
        # Upload para staging
        print(f"\n[BIGQUERY] Enviando {len(df_to_upload)} registros para staging: '{staging_table_id}'...")
        pandas_gbq.to_gbq(
            df_to_upload,
            destination_table=staging_table_id,
            project_id=project_id,
            if_exists='replace',
            table_schema=table_schema
        )
        print("[BIGQUERY] ✓ Upload para staging concluído")

        # Executar MERGE
        print(f"\n[BIGQUERY] Executando MERGE para tabela principal: '{main_table_id}'...")

        client = bigquery.Client(project=project_id)

        merge_sql = f"""
        MERGE `{project_id}.{main_table_id}` T
        USING `{project_id}.{staging_table_id}` S
        ON T.link = S.link

        -- Se o link já existe MAS a data_publicacao está NULA, atualize-a
        WHEN MATCHED AND T.data_publicacao IS NULL THEN
          UPDATE SET T.data_publicacao = S.data_publicacao

        -- Se o link é totalmente novo, insira a linha
        WHEN NOT MATCHED BY TARGET THEN
          INSERT (fonte, titulo, resumo, link, horario_coleta, data_publicacao)
          VALUES(S.fonte, S.titulo, S.resumo, S.link, S.horario_coleta, S.data_publicacao)
        """

        query_job = client.query(merge_sql)
        query_job.result()

        print(f"[BIGQUERY] ✓ MERGE concluído! {query_job.num_dml_affected_rows} linhas afetadas")
        print("\n=== [BIGQUERY] SINCRONIZAÇÃO FINALIZADA COM SUCESSO ===")

    except Exception as e:
        print(f"\n[BIGQUERY] ❌ ERRO: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n[BIGQUERY] Nenhum dado para sincronizar.")


=== [BIGQUERY] INICIANDO SINCRONIZAÇÃO ===
--- [AUTH] Autenticando para acesso ao BigQuery ---
--- [AUTH] Autenticação concluída ---

[BIGQUERY] Enviando 5 registros para staging: 'scrapping.dc_piracicaba_staging'...


100%|██████████| 1/1 [00:00<00:00, 8701.88it/s]


[BIGQUERY] ✓ Upload para staging concluído

[BIGQUERY] Executando MERGE para tabela principal: 'scrapping.dc_piracicaba'...

[BIGQUERY] ❌ ERRO: 404 Not found: Table bacias-pcj:scrapping.dc_piracicaba was not found in location US; reason: notFound, message: Not found: Table bacias-pcj:scrapping.dc_piracicaba was not found in location US

Location: US
Job ID: 011f5064-8b93-4ce7-beee-47001f1a16bd



Traceback (most recent call last):
  File "/tmp/ipython-input-235214199.py", line 66, in <cell line: 0>
    query_job.result()
  File "/usr/local/lib/python3.12/dist-packages/google/cloud/bigquery/job/query.py", line 1773, in result
    while not is_job_done():
              ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_base.py", line 214, in _retry_error_helper
    raise final_exc from source_exc
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 147, in retry_target
    result = target()
             ^^^^^^^^
  File "/usr/local/lib/python3.12/dist-p